**Basic SQL Reminder Notes**

SELECT *

FROM *

WHERE *

"%" is wildcard for strings

"XOR" for one or other, but not both

ORDER BY [column name] asc/desc (asc/desc can be used after individual columns

ROUND works as we understand
--------

IN for lists Example:
-----
SELECT name FROM bbc
  WHERE name IN ('Sri Lanka', 'Ceylon',
                 'Persia',    'Iran')


**Multiple, indepnedent clauses are possible. Note the parentheses**

SELECT * FROM nobel

  WHERE subject IN ('physics')
  
  AND yr IN (1980)
  
  OR
  
  (subject IN ('chemistry')
  AND yr IN (1984))

ERD - Entity Relationship Diagram

In [13]:
%reload_ext sql
%sql sqlite:///test_capital_project.db

'Connected: @test_capital_project.db'

In [28]:
%%sql

DROP TABLE IF EXISTS balance_sheets;
DROP TABLE IF EXISTS business_units;
DROP TABLE IF EXISTS capital_buffers;
DROP TABLE IF EXISTS forecast_assumptions;

CREATE TABLE business_units (
    unit_id INTEGER PRIMARY KEY,
    unit_name TEXT NOT NULL
);

CREATE TABLE balance_sheets (
    entry_id INTERGER PRIMARY KEY,
    unit_id INTEGER NOT NULL,
    report_date DATE NOT NULL,
    assets FLOAT NOT NULL,
    liabilities FLOAT NOT NULL,
    FOREIGN KEY (unit_id) REFERENCES business_units(unit_id)
);

CREATE TABLE capital_buffers (
    buffer_id INTEGER PRIMARY KEY,
    unit_id INTEGER NOT NULL,
    min_capital_ratio FLOAT NOT NULL,
    FOREIGN KEY (unit_id) REFERENCES business_units(unit_id)
);

CREATE TABLE forecast_assumptions (
    assumption_id INTEGER PRIMARY KEY,
    scenario_name TEXT NOT NULL,
    asset_multiplier FLOAT NOT NULL,
    liab_multiplier FLOAT NOT NULL
);

 * sqlite:///test_capital_project.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [33]:
%%sql
INSERT INTO business_units (unit_id, unit_name) VALUES
(1, 'Retail Banking'),
(2, 'Commercial Lending'),
(3, 'Credit Cards'),
(4, 'Wealth Management');

 * sqlite:///test_capital_project.db
4 rows affected.


[]

In [34]:
%%sql
SELECT * FROM business_units

 * sqlite:///test_capital_project.db
Done.


unit_id,unit_name
1,Retail Banking
2,Commercial Lending
3,Credit Cards
4,Wealth Management


In [ ]:
import pandas as pd

result = %sql SELECT * FROM balance_sheets
df = result.DataFrame()
df.head()